In [1]:
%pip install wildlife-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wildlife-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install timm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [5]:
import itertools
from itertools import chain
import timm
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset

from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer, TripletLoss

2024-08-06 03:36:16.611150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 03:36:16.611293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 03:36:16.783009: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**IPanda50 Dataset**

In [6]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()


DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=0848569d-5f33-46f1-9f6c-9e5f39a3abaa
To: /kaggle/data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:07<00:00, 122MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /kaggle/data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 73.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /kaggle/data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 184MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

...

Epoch 18: Average Loss = 4.991553783416748


Epoch 18: 100%|█████████████████████████████████████████████████████| 54/54 [01:31<00:00,  1.69s/it]


Epoch 19: Average Loss = 4.072727680206299


Epoch 19: 100%|█████████████████████████████████████████████████████| 54/54 [01:31<00:00,  1.69s/it]

Epoch 20: Average Loss = 3.6490542888641357


In [7]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_IPanda50_retrained_checkpoint.pth")

In [8]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [9]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 53/53 [01:00<00:00,  1.14s/it]


In [10]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-2.1417500e-02, -6.2752441e-02, -3.7886046e-02, ...,
         8.0639780e-01, -2.3861703e-02, -2.2734758e-02],
       [ 5.5107418e-03, -5.5852152e-02, -1.4979074e-02, ...,
         3.0285709e-02, -1.3994721e-01, -3.3173028e-02],
       [-5.3321950e-02, -1.6287115e-01,  4.4074226e-03, ...,
        -5.0728880e-02, -7.8984678e-02, -6.7862809e-02],
       ...,
       [-8.4366478e-02, -5.8164906e-02,  2.4536673e-02, ...,
        -1.2659199e-02, -8.9036934e-02, -2.8803427e-02],
       [-3.4749683e-02,  2.9800681e-02, -9.8843738e-02, ...,
         3.3994600e-02,  1.4757553e-01,  2.5087292e-04],
       [-3.3111785e-02, -2.7916227e-03,  9.1598958e-01, ...,
         2.4535820e-02, -4.1479819e-02, -3.9219651e-02]], dtype=float32)}


In [11]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['36_xingda' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '28_sa' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '36_xingda' '34_xiaoqiao' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '36_xingda' '28_sa' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '04_chengdui' '35_xilan' '08_fulai' '00_aibang' '28_sa' '29_shuangxiong'
 '46_yazhu' '44_yayi' '14_maodou' '30_shuqing' '36_xingda' '39_xinghui'
 '33_wuyi' '10_hexing' '10_hexing' '00_aibang' '49_yuanrun' '35_xilan'
 '04_chengdui' '10_hexing' '39_xinghui' '36_xingda' '37_xinger'
 '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui' '10_hexing'
 '17_meibang' '00_aibang' '28_sa' '14_maodou' '15_maosun' '32_susu'
 '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomiao' '48_yongba

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [12]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))

Accuracy on IPanda50 data: 100.00%


In [13]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [14]:
# # Assuming you need to use the model for further operations
# # Initialize the model architecture again as needed for loading
# # Make sure to initialize it exactly as you did for training
# model = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=False)

# # # Create a BasicTrainer instance with the initialized model (other parameters should match the training setup)
# # trainer = BasicTrainer(
# #     dataset=None,  # Assuming no dataset is needed just for loading
# #     model=model,
# #     objective=None,  # Assuming no objective needed just for loading
# #     optimizer=None,  # Assuming no optimizer needed just for loading
# #     epochs=0,  # No training epochs needed for just loading
# #     device='cuda'  # Adjust as per your device configuration
# # )

# trainer = BasicTrainer(
#     dataset=dataset,
#     model=backbone,
#     objective=objective,
#     optimizer=optimizer,
#     epochs=2,
#     device='cuda',
#     epoch_callback=print_epoch_loss
# )
# # Load the model
# trainer.load("retrained_chks/arcfaceloass_retrained_checkpoint.pth")

# # After loading, the model in the trainer instance is now the re-trained model
# # and is ready for use in further processing or inference


**MacaqueFaces Dataset**

In [15]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')
# Load dataset metadata
metadata = datasets.MacaqueFaces('../data/MacaqueFaces')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET MacaqueFaces: DOWNLOADING STARTED.


MacaqueFaces.zip: 12.0MB [00:00, 13.9MB/s]                            
MacaqueFaces_ImageInfo.csv: 410kB [00:00, 1.02MB/s]                            


DATASET MacaqueFaces: EXTRACTING STARTED.
DATASET MacaqueFaces: FINISHED.



Epoch 0:   0%|                                                               | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|████████████████████████████████████████████████████▉ | 49/50 [01:22<00:01,  1.66s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 50/50 [01:22<00:00,  1.66s/it]


Epoch 1: Average Loss = 24.94143295288086


...

Epoch 18: Average Loss = 0.5163429975509644


Epoch 18: 100%|█████████████████████████████████████████████████████| 50/50 [01:22<00:00,  1.65s/it]


Epoch 19: Average Loss = 0.49987325072288513


Epoch 19: 100%|█████████████████████████████████████████████████████| 50/50 [01:22<00:00,  1.65s/it]

Epoch 20: Average Loss = 0.5094174742698669


In [16]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_MacaqueFaces_retrained_checkpoint.pth")

In [17]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 49/49 [00:29<00:00,  1.66it/s]


In [18]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[0.91731256, 0.9817444 , 0.971934  , ..., 0.0368656 , 0.03633226,
        0.04372552],
       [0.9326054 , 0.98513794, 0.9802152 , ..., 0.07727763, 0.0784016 ,
        0.0861953 ],
       [0.93377304, 0.99035513, 0.98925805, ..., 0.04479201, 0.04619441,
        0.05449455],
       ...,
       [0.9321706 , 0.99156415, 0.989763  , ..., 0.04636301, 0.04647823,
        0.05706368],
       [0.9224577 , 0.98855805, 0.98566794, ..., 0.04969281, 0.04629517,
        0.05735105],
       [0.94088566, 0.9828846 , 0.97803926, ..., 0.10768093, 0.10712467,
        0.11637452]], dtype=float32)}


In [19]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [20]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_P * 100))

Accuracy on MacaqueFaces data: 100.00%


In [21]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**LionData Dateset**

In [22]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')
# Load dataset metadata
metadata = datasets.LionData('../data/LionData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET LionData: DOWNLOADING STARTED.


main.zip: 495MB [00:20, 24.6MB/s]


DATASET LionData: EXTRACTING STARTED.
DATASET LionData: FINISHED.



Epoch 0:   0%|                                                                | 0/6 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.52s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.63s/it]


Epoch 1: Average Loss = 36.8021354675293


...

Epoch 18: Average Loss = 33.16015625


Epoch 18: 100%|███████████████████████████████████████████████████████| 6/6 [00:10<00:00,  1.69s/it]


Epoch 19: Average Loss = 32.88702392578125


Epoch 19: 100%|███████████████████████████████████████████████████████| 6/6 [00:10<00:00,  1.69s/it]

Epoch 20: Average Loss = 32.554256439208984


In [23]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_LionData_retrained_checkpoint.pth")

In [24]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [25]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.46it/s]


In [26]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[0.30619138, 0.3194886 , 0.49256814, ..., 0.611298  , 0.46321848,
        0.3901457 ],
       [0.6390827 , 0.2937554 , 0.62275714, ..., 0.300912  , 0.59419376,
        0.37286848],
       [0.22977239, 0.32008484, 0.40241268, ..., 0.44615704, 0.5013564 ,
        0.6091844 ],
       ...,
       [0.22411226, 0.40842342, 0.3827599 , ..., 0.68785405, 0.55094326,
        0.5227213 ],
       [0.11210939, 0.2982122 , 0.42886007, ..., 0.5044739 , 0.6140084 ,
        0.68350255],
       [0.23061556, 0.41766965, 0.39014614, ..., 0.60067624, 0.51764905,
        0.55700004]], dtype=float32)}


In [27]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Amber' 'Mooza' 'Shambe' 'Tigisi' 'Nashipai' 'Moswen' 'Enderoni'
 'Simaloi' 'Spring' 'Joy' 'Maskio' 'Kioni' 'Nuru' 'Kinna' 'Autumn' 'Lilly'
 'Spring' 'Kibibi' 'Enderoni' 'Saitoti' 'Senteu' 'Enkume' 'Safi'
 'Olbarnoti' 'Chaimu' 'Romeo' 'Ayo' 'Naisiae' 'Sadala' 'Mashavu' 'Moswen'
 'Napi' 'Arria' 'Kinna' 'Napi' 'Kibibi' 'Naengop' 'Kioni' 'Leia' 'Kibibi'
 'Lucinda' 'Shambe' 'Leia' 'Sebastian' 'Olonyori' 'Naisiae' 'Mooza'
 'Moswen' 'Ngare' 'Naini' 'Kioni' 'Lucinda' 'Kinna' 'Kioni' 'Sadala'
 'Mashavu' 'Safi' 'Mama-Kali' 'Kioni' 'Napi' 'Opi' 'Blackie' 'Kibibi'
 'Doto' 'Nariku-Inkgera' 'White-Eye' 'Nakepari' 'Bibi' 'Joy' 'Amber'
 'Shemsa' 'Olonyori' 'Mickey' 'Esiriwua' 'Tigisi' 'Bibi' 'Blackie' 'Safi'
 'Lemayian' 'Nariku-Inkgera' 'Cleopatra' 'Mashavu' 'Mama-Kali' 'Lolparpit'
 'Lucinda' 'Naini' 'Rescue' 'Moswen' 'Empurra' 'Leia' 'Blackie' 'Simaloi'
 'Lucinda' 'Cleopatra' 'Selenkay' 'Doto' 'Maskio' 'Enkume' 'Naape'
 'Enderoni']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [28]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on LionData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on LionData data: 20.00%


In [29]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.6016666666666667
Recall: 0.2
F1 Score: 0.1749047619047619


**NyalaDataSet**

In [30]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')
# Load dataset metadata
metadata = datasets.NyalaData('../data/NyalaData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET NyalaData: DOWNLOADING STARTED.


main.zip: 495MB [00:19, 25.3MB/s]


DATASET NyalaData: EXTRACTING STARTED.
DATASET NyalaData: FINISHED.



Epoch 0:   0%|                                                               | 0/16 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▋   | 15/16 [00:49<00:03,  3.15s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.13s/it]


Epoch 1: Average Loss = 37.456298828125


...

Epoch 18: Average Loss = 32.66950225830078


Epoch 18: 100%|█████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.16s/it]


Epoch 19: Average Loss = 32.31733703613281


Epoch 19: 100%|█████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.14s/it]

Epoch 20: Average Loss = 31.866565704345703


In [31]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_NyalaData_retrained_checkpoint.pth")

In [32]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 15/15 [00:46<00:00,  3.09s/it]


In [33]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.33430412, -0.04669978, -0.11801578, ...,  0.06442522,
        -0.06267003,  0.30298132],
       [ 0.14868379,  0.493905  ,  0.28760958, ...,  0.18865052,
         0.27258238,  0.13491218],
       [ 0.11450575,  0.30070543,  0.28517532, ...,  0.8053575 ,
         0.16371356,  0.25192255],
       ...,
       [-0.04791659,  0.2854673 ,  0.23032412, ...,  0.09722105,
         0.34015295,  0.0233671 ],
       [ 0.19964384,  0.06520479,  0.13312519, ...,  0.2645728 ,
         0.09535416,  0.3753656 ],
       [-0.07508553,  0.34507418,  0.60637677, ...,  0.15972188,
         0.59034646,  0.03071071]], dtype=float32)}


In [34]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['155' '10' '57' '149' '144' '102' '147' '56' '114' '100' '87' '51' '33'
 '21' '193' '123' '36' '27' '131' '68' '85' '68' '104' '30' '10' '8' '48'
 '67' '34' '46' '145' '27' '147' '166' '23' '140' '124' '32' '12' '19'
 '51' '166' '30' '46' '241' '241' '114' '67' '21' '149' '115' '147' '243'
 '108' '223' '181' '104' '252' '32' '247' '46' '121' '40' '85' '31' '210'
 '65' '242' '11' '21' '142' '12' '60' '164' '102' '166' '76' '124' '51'
 '36' '20' '149' '212' '85' '42' '32' '158' '71' '165' '36' '3' '84' '21'
 '246' '33' '41' '29' '85' '40' '89']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [35]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on NyalaData data: 45.00%


In [36]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.8033333333333332
Recall: 0.45
F1 Score: 0.45719047619047615


**StripeSpotter Dataset**

In [37]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')
# Load dataset metadata
metadata = datasets.StripeSpotter('../data/StripeSpotter')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET StripeSpotter: DOWNLOADING STARTED.


data-20110718.zip: 71.7MB [00:01, 59.2MB/s]                            
data-20110718.z02: 100%|██████████| 78.6M/78.6M [00:01<00:00, 48.2MB/s]
data-20110718.z01: 100%|██████████| 78.6M/78.6M [00:01<00:00, 42.4MB/s]


DATASET StripeSpotter: EXTRACTING STARTED.
 copying: data/
 copying: data/.DS_Store
 copying: data/images/
 copying: data/images/img-0000001.jpg
 copying: data/images/img-0000002.jpg
 copying: data/images/img-0000003.jpg
 copying: data/images/img-0000004.jpg
 copying: data/images/img-0000005.jpg
 copying: data/images/img-0000006.jpg
 copying: data/images/img-0000007.jpg
 copying: data/images/img-0000008.jpg
 copying: data/images/img-0000009.jpg
 copying: data/images/img-0000010.jpg
 copying: data/images/img-0000011.jpg
 copying: data/images/img-0000012.jpg
 copying: data/images/img-0000013.jpg
 copying: data/images/img-0000014.jpg
 copying: data/images/img-0000015.jpg
 copying: data/images/img-0000016.jpg
 copying: data/images/img-0000017.jpg
 copying: data/images/img-0000018.jpg
 copying: data/images/img-0000019.jpg
 copying: data/images/img-0000020.jpg
 copying: data/images/img-0000021.jpg
 copying: data/images/img-0000022.jpg
 copying: data/images/img-0000023.jpg
 copying: data/imag

Epoch 0:   0%|                                                                | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  86%|████████████████████████████████████████████████        | 6/7 [00:23<00:03,  3.67s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 7/7 [00:24<00:00,  3.44s/it]


Epoch 1: Average Loss = 35.538238525390625


...

Epoch 18: Average Loss = 6.58345365524292


Epoch 18: 100%|███████████████████████████████████████████████████████| 7/7 [00:24<00:00,  3.45s/it]


Epoch 19: Average Loss = 5.542449474334717


Epoch 19: 100%|███████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.41s/it]

Epoch 20: Average Loss = 5.253093719482422


In [38]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_StripeSpotter_retrained_checkpoint.pth")

In [39]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [40]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.48s/it]


In [41]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-0.01482271,  0.02251977,  0.05265947, ...,  0.0327915 ,
        -0.08021441, -0.03684006],
       [ 0.10741799,  0.05535059,  0.13566074, ...,  0.840462  ,
         0.18438563, -0.09881208],
       [ 0.19470608,  0.09089143, -0.02702202, ..., -0.00682867,
        -0.04842671, -0.03993186],
       ...,
       [ 0.19450995,  0.18415596, -0.05738789, ...,  0.00270942,
         0.12889415, -0.01274869],
       [ 0.10239202,  0.0329514 , -0.17682053, ..., -0.03541449,
         0.01095304, -0.0695518 ],
       [ 0.00452232, -0.0351746 , -0.03980768, ...,  0.08968559,
        -0.03407484, -0.06353452]], dtype=float32)}


In [42]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['09_212' 'S09_161' '08_085' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' 'S09_067' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '09_212' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '09_212'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 'OP05_625' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679'
 'S09_111' '01_679' '01_036' 'S09_067']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [43]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_P * 100))

Accuracy on StripeSpotter data: 99.00%


In [44]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.992
Recall: 0.99
F1 Score: 0.9855555555555555


**CZoo Dataset**

In [45]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')
# Load dataset metadata
metadata = datasets.CZoo('../data/CZoo')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CZoo: DOWNLOADING STARTED.


master.zip: 634MB [00:19, 32.1MB/s]


DATASET CZoo: EXTRACTING STARTED.
DATASET CZoo: FINISHED.



Epoch 0:   0%|                                                               | 0/17 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▊   | 16/17 [00:28<00:01,  1.75s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 17/17 [00:29<00:00,  1.73s/it]


Epoch 1: Average Loss = 32.347801208496094


...

Epoch 18: Average Loss = 0.5814058184623718


Epoch 18: 100%|█████████████████████████████████████████████████████| 17/17 [00:28<00:00,  1.68s/it]


Epoch 19: Average Loss = 0.5263033509254456


Epoch 19: 100%|█████████████████████████████████████████████████████| 17/17 [00:28<00:00,  1.70s/it]

Epoch 20: Average Loss = 0.4195416569709778


In [46]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_CZoo_retrained_checkpoint.pth")

In [47]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 16/16 [00:15<00:00,  1.00it/s]


In [48]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.9278866 ,  0.93542147,  0.9249872 , ...,  0.05817145,
        -0.04902707,  0.0166591 ],
       [ 0.9573866 ,  0.9496364 ,  0.9387225 , ...,  0.07305826,
        -0.03971243,  0.00935403],
       [ 0.94516134,  0.95462775,  0.93582535, ...,  0.0483136 ,
        -0.04772014,  0.02035457],
       ...,
       [ 0.9569787 ,  0.97808146,  0.9634459 , ...,  0.04226884,
        -0.04084624,  0.04665694],
       [ 0.9647658 ,  0.9765996 ,  0.96618026, ...,  0.04295793,
        -0.05440514,  0.04071505],
       [ 0.9610641 ,  0.9791463 ,  0.96649003, ...,  0.03189659,
        -0.04389849,  0.03158419]], dtype=float32)}


In [49]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Alex' 'Alex' 'Alex' 'Jahaga' 'Jahaga' 'Alex' 'Alex' 'Jahaga' 'Alex'
 'Jahaga' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex'
 'Alex' 'Alex' 'Alex' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Gertrudia' 'Gertrudia' 'Gertrudia' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Annett' 'Annett' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Al

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [50]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on Czoo data: {:.2f}%".format(accuracy_P * 100))

Accuracy on Czoo data: 100.00%


In [51]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**CowDataset**

In [52]:
# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')
# Load dataset metadata
metadata = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CowDataset: DOWNLOADING STARTED.


31210192: 4.45GB [02:59, 24.8MB/s]                            


DATASET CowDataset: EXTRACTING STARTED.
DATASET CowDataset: FINISHED.



Epoch 0:   0%|                                                               | 0/12 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:55<00:00, 17.63s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:55<00:00, 19.62s/it]


Epoch 1: Average Loss = 32.5689582824707


...

Epoch 18: Average Loss = 0.27882352471351624


Epoch 18: 100%|█████████████████████████████████████████████████████| 12/12 [03:41<00:00, 18.46s/it]


Epoch 19: Average Loss = 0.34006571769714355


Epoch 19: 100%|█████████████████████████████████████████████████████| 12/12 [03:31<00:00, 17.62s/it]

Epoch 20: Average Loss = 0.21366126835346222


In [53]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_CowDataset_retrained_checkpoint.pth")

In [54]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 11/11 [03:38<00:00, 19.82s/it]


In [55]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-0.08821815, -0.01579853, -0.03447824, ...,  0.04288876,
        -0.01770647, -0.01339302],
       [-0.08671182, -0.05336584, -0.17106716, ..., -0.18728513,
         0.06460956, -0.00323403],
       [-0.04771869, -0.00236316, -0.04375275, ...,  0.06956739,
        -0.04608474, -0.0469218 ],
       ...,
       [-0.12251632, -0.03238332, -0.04642313, ...,  0.06367704,
        -0.00817465, -0.00484858],
       [ 0.05575105, -0.20818332,  0.04779074, ..., -0.03088545,
        -0.06632924, -0.06450051],
       [-0.09239939, -0.08780532, -0.01422473, ..., -0.16310996,
         0.9376149 ,  0.9442916 ]], dtype=float32)}


In [56]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '5' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1' '2'
 '5' '5' '9' '4' '1' '10' '3' '11' '12' '5' '3' '6' '5' '5' '2' '7' '11'
 '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '6' '3' '8' '10'
 '11' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [57]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on CowDataset data: {:.2f}%".format(accuracy_P * 100))

Accuracy on CowDataset data: 100.00%


In [58]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


 # **On New Unseen Dataset**

**DogFaceNet Dataset**

In [59]:

# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata = datasets.DogFaceNet('../data/DogFaceNet')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET DogFaceNet: DOWNLOADING STARTED.


DogFaceNet_Dataset_224_1.zip: 75.6MB [00:01, 68.9MB/s]                            


DATASET DogFaceNet: EXTRACTING STARTED.
DATASET DogFaceNet: FINISHED.



Epoch 0:   0%|                                                               | 0/66 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|█████████████████████████████████████████████████████▏| 65/66 [01:48<00:01,  1.65s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 66/66 [01:49<00:00,  1.65s/it]


Epoch 1: Average Loss = 39.3543586730957


...

Epoch 18: Average Loss = 28.51412582397461


Epoch 18: 100%|█████████████████████████████████████████████████████| 66/66 [01:50<00:00,  1.67s/it]


Epoch 19: Average Loss = 27.900630950927734


Epoch 19: 100%|█████████████████████████████████████████████████████| 66/66 [01:49<00:00,  1.66s/it]

Epoch 20: Average Loss = 27.2148494720459


In [60]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_DogFaceNet_retrained_checkpoint.pth")

In [61]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [62]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 65/65 [00:39<00:00,  1.64it/s]


In [63]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-0.19835502, -0.00824073,  0.03312841, ...,  0.08426014,
         0.0105888 , -0.08770564],
       [-0.04444366, -0.05371714,  0.0981549 , ..., -0.07689315,
        -0.0124583 , -0.04216375],
       [-0.06403714, -0.05218634,  0.00577466, ...,  0.11319457,
         0.02078415,  0.01869084],
       ...,
       [-0.04116895, -0.03853647,  0.05523988, ..., -0.06938845,
        -0.03596522,  0.08614846],
       [-0.0715552 ,  0.11094338,  0.00110462, ..., -0.09182444,
         0.04303921,  0.04647809],
       [-0.06242361,  0.02907601, -0.05520296, ..., -0.00641352,
         0.08603434, -0.04300848]], dtype=float32)}


In [64]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['1223' '182' '919' '1122' '428' '378' '1063' '426' '470' '405' '764'
 '880' '671' '1339' '432' '673' '1075' '274' '296' '234' '1039' '991'
 '485' '425' '280' '407' '858' '1248' '1232' '896' '892' '448' '240' '251'
 '448' '930' '223' '210' '345' '763' '622' '1324' '158' '127' '961' '1254'
 '1139' '208' '513' '424' '293' '415' '825' '147' '373' '192' '333' '333'
 '1314' '957' '329' '89' '189' '100' '122' '631' '367' '646' '26' '903'
 '621' '673' '174' '474' '1169' '458' '473' '149' '373' '622' '540' '919'
 '589' '913' '782' '1357' '361' '417' '1324' '953' '183' '200' '299' '191'
 '397' '187' '270' '563' '1020' '1074']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [65]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on DogFaceNet data: {:.2f}%".format(accuracy_P * 100))

Accuracy on DogFaceNet data: 90.00%


In [66]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 0.9
F1 Score: 0.9033333333333334
